In [1]:
import pandas as pd

In [2]:
# initialise data of lists. 
data1 = {'Name':['Tom inc.', 'Jack AG', 'Nick DE', 'Julianna inc.'], 
        'marks':[99, 98, 95, 90]} 

data2 = {'Name':['Tommy international', 'Jacky inc.', 'Nickelson company', 'Julian mnc'], 
        'Age':[20, 21, 19, 18]} 

In [3]:
# Creating Dataframes
df1 = pd.DataFrame(data1) 
df2 = pd.DataFrame(data2) 

In [4]:
# checking data
df1

,Name,marks
0,Tom inc.,99
1,Jack AG,98
2,Nick DE,95
3,Julianna inc.,90


In [5]:
# checking data
df2

,Name,Age
0,Tommy international,20
1,Jacky inc.,21
2,Nickelson company,19
3,Julian mnc,18


The above mentioned matches are really in a bad shape and they all require more cleaning to be matched with any kind of algorithm. Now we will do some name cleanings first in order to match the names perfectly.

In [9]:
# Importing regular expressions
import re

In [12]:
# Lets try the strings methods to manipulate the names in both the dataframes using regular expression

def cleanData(series):
    return series.str.lower().str.replace(r"[^a-z\s]", "").str.replace(r"\b(the|corporation|international|company|corp|comp|mnc|inc|ag|de)\b", "").str.strip()     

In the above line code the ^a-z means that we dont need anything that falls between a-z. Further on we are removing the strings like international inc. corporation company mnc etc. For this we need to keep in mind that longer strings need to come first otherwise if the shorter string gets removed first and it is the subset of the longer string than the rest of string remains there and that would be really confusing. In the regular expression the second replacement of string we have completed the word boundry by keeping the words in brackets and putting \b in the ends.

In [13]:
cleanData(df1['Name'])

0         tom
1        jack
2        nick
3    julianna
Name: Name, dtype: object

In [14]:
cleanData(df2['Name'])

0        tommy
1        jacky
2    nickelson
3       julian
Name: Name, dtype: object

In [15]:
# Now we will create columns on the basis of cleaned names in both the dataframes

df1["CleanNames"] = cleanData(df1['Name'])

In [16]:
df2["CleanNames"] = cleanData(df2['Name'])

In [17]:
df1

,Name,marks,CleanNames
0,Tom inc.,99,tom
1,Jack AG,98,jack
2,Nick DE,95,nick
3,Julianna inc.,90,julianna


In [18]:
df2

,Name,Age,CleanNames
0,Tommy international,20,tommy
1,Jacky inc.,21,jacky
2,Nickelson company,19,nickelson
3,Julian mnc,18,julian


In [19]:
# importing fuzzy pandas library
# installation command !pip install fuzzy_pandas

import fuzzy_pandas as fpd

In [28]:
#checking the matches of the company

matches = fpd.fuzzy_merge(df1, df2, 
                          left_on = ['CleanNames'], 
                          right_on = ['CleanNames'], 
                          ignore_case = True,
                          # keep = 'match'
                          # method = 'levenshtein',
                          method = 'jaro',
                          threshold = 0.8)

In [29]:
# printing the matches, for long list of dataframe use df.head(number) command to check more results. Tweak the
# threshold value in the above cell to make it loose or tight string matching.
print(matches)

            Name  marks CleanNames                 Name  Age CleanNames
0       Tom inc.     99        tom  Tommy international   20      tommy
1        Jack AG     98       jack           Jacky inc.   21      jacky
2        Nick DE     95       nick    Nickelson company   19  nickelson
3  Julianna inc.     90   julianna           Julian mnc   18     julian


In [31]:
# Lets create a database of the above results

dfResult = pd.DataFrame(matches)

In [33]:
dfMatch = dfResult[['Name', 'CleanNames', 'Age', 'marks']]

In [34]:
dfMatch

,Name,Name,CleanNames,CleanNames,Age,marks
0,Tom inc.,Tommy international,tom,tommy,20,99
1,Jack AG,Jacky inc.,jack,jacky,21,98
2,Nick DE,Nickelson company,nick,nickelson,19,95
3,Julianna inc.,Julian mnc,julianna,julian,18,90
